<a href="https://colab.research.google.com/github/Paul2709/2024UCALCD001/blob/main/TP_Informatica_General.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import csv

CATEGORIAS = ["Comida", "Transporte", "Vivienda", "Entretenimiento", "Otros"]
TRANSACCIONES = "transacciones.txt"

def agregar_transaccion():
    """
    Agrega una nueva transacción al archivo de transacciones.
    """
    while True:
        tipo = input("Ingresa el tipo de transacción (gasto/ingreso): ").lower()
        if tipo in ["gasto", "ingreso"]:
            break
        print("Tipo de transacción inválido")

    while True:
        categoria = input(f"Ingresa la categoría ({', '.join(CATEGORIAS)}): ").capitalize()
        if categoria in CATEGORIAS:
            break
        print("Categoría inválida")

    descripcion = input("Ingresa una descripción para la transacción: ")

    while True:
        try:
            monto = float(input("Ingresa el monto de la transacción: "))
            if monto > 0:
                break
            print("El monto debe ser un valor positivo")
        except ValueError:
            print("Monto inválido")

    while True:
        fecha = input("Ingresa la fecha de la transacción (dd/mm/aaaa): ")
        if validar_fecha(fecha):
            break
        print("Fecha inválida")

    with open(TRANSACCIONES, "a") as archivo:
        archivo.write(f"{tipo},{categoria},{descripcion},{monto},{fecha}\n")
    print(f"Se ha registrado un {tipo} de {monto} en la categoría '{categoria}' por '{descripcion}' el {fecha}")

def listar_transacciones():
    """
    Imprime todas las transacciones registradas en el archivo de transacciones.
    """
    if not os.path.isfile(TRANSACCIONES):
        print("No hay transacciones registradas.")
        return

    print("Transacciones registradas:")
    with open(TRANSACCIONES, "r") as archivo:
        for i, linea in enumerate(archivo, start=1):
            tipo, categoria, descripcion, monto, fecha = linea.strip().split(",")
            print(f"{i}. {fecha} - {tipo.capitalize()} de {monto} en '{categoria}' por '{descripcion}'")

def resumen_finanzas():
    """
    Muestra un resumen de los gastos e ingresos registrados.
    """
    gastos = {}
    ingresos = 0

    if not os.path.isfile(TRANSACCIONES):
        print("No hay transacciones registradas.")
        return

    with open(TRANSACCIONES, "r") as archivo:
        for linea in archivo:
            tipo, categoria, _, monto, _ = linea.strip().split(",")
            monto = float(monto)
            if tipo == "gasto":
                gastos[categoria] = gastos.get(categoria, 0) + monto
            else:
                ingresos += monto

    print("Resumen de finanzas:")
    print("Gastos:")
    for categoria, gasto in gastos.items():
        print(f"- {categoria}: {gasto}")
    print(f"Total gastos: {sum(gastos.values())}")
    print(f"Total ingresos: {ingresos}")

def editar_transaccion():
    """
    Permite editar una transacción existente.
    """
    listar_transacciones()

    while True:
        try:
            indice = int(input("Ingresa el número de la transacción que deseas editar: "))
            break
        except ValueError:
            print("Índice inválido")

    transacciones = leer_transacciones()
    if indice < 1 or indice > len(transacciones):
        print("Índice fuera de rango")
        return

    transaccion = transacciones[indice - 1]
    tipo, categoria, descripcion, monto, fecha = transaccion.split(",")

    while True:
        nuevo_tipo = input(f"Ingresa el nuevo tipo de transacción (actual: {tipo}) o déjalo en blanco para mantener: ").lower()
        if nuevo_tipo in ["gasto", "ingreso", ""]:
            break
        print("Tipo de transacción inválido")

    while True:
        nueva_categoria = input(f"Ingresa la nueva categoría (actual: {categoria}) o déjalo en blanco para mantener: ").capitalize()
        if nueva_categoria in CATEGORIAS or nueva_categoria == "":
            break
        print("Categoría inválida")

    nueva_descripcion = input(f"Ingresa la nueva descripción (actual: {descripcion}) o déjalo en blanco para mantener: ")
    nueva_descripcion = nueva_descripcion if nueva_descripcion else descripcion

    while True:
        try:
            nuevo_monto = float(input(f"Ingresa el nuevo monto (actual: {monto}) o déjalo en blanco para mantener: "))
            break
        except ValueError:
            if input("Monto inválido. ¿Deseas mantener el monto actual? (s/n) ").lower() == "s":
                nuevo_monto = float(monto)
                break

    while True:
        nueva_fecha = input(f"Ingresa la nueva fecha (actual: {fecha}) o déjalo en blanco para mantener: ")
        if nueva_fecha == "" or validar_fecha(nueva_fecha):
            break
        print("Fecha inválida")

    nueva_transaccion = ",".join([nuevo_tipo if nuevo_tipo else tipo,
                                  nueva_categoria if nueva_categoria else categoria,
                                  nueva_descripcion,
                                  str(nuevo_monto) if nuevo_monto else monto,
                                  nueva_fecha if nueva_fecha else fecha])

    transacciones[indice - 1] = nueva_transaccion
    escribir_transacciones(transacciones)
    print("Transacción editada correctamente.")

def eliminar_transaccion():
    """
    Permite eliminar una transacción existente.
    """
    listar_transacciones()

    while True:
        try:
            indice = int(input("Ingresa el número de la transacción que deseas eliminar: "))
            break
        except ValueError:
            print("Índice inválido")

    transacciones = leer_transacciones()
    if indice < 1 or indice > len(transacciones):
        print("Índice fuera de rango")
        return

    transaccion = transacciones.pop(indice - 1)
    escribir_transacciones(transacciones)
    print(f"Transacción eliminada: {transaccion}")

def exportar_csv():
    """
    Exporta las transacciones registradas a un archivo CSV.
    """
    if not os.path.isfile(TRANSACCIONES):
        print("No hay transacciones registradas.")
        return

    with open(TRANSACCIONES, "r") as archivo_entrada, open("transacciones.csv", "w", newline="") as archivo_salida:
        escritor_csv = csv.writer(archivo_salida)
        escritor_csv.writerow(["Tipo", "Categoría", "Descripción", "Monto", "Fecha"])
        for linea in archivo_entrada:
            tipo, categoria, descripcion, monto, fecha = linea.strip().split(",")
            escritor_csv.writerow([tipo, categoria, descripcion, monto, fecha])

    print("Transacciones exportadas correctamente a 'transacciones.csv'")

def leer_transacciones():
    """
    Lee las transacciones del archivo de transacciones y las devuelve como una lista.
    """
    transacciones = []
    if os.path.isfile(TRANSACCIONES):
        with open(TRANSACCIONES, "r") as archivo:
            for linea in archivo:
                transacciones.append(linea.strip())
    return transacciones

def escribir_transacciones(transacciones):
    """
    Escribe las transacciones en el archivo de transacciones.
    """
    with open(TRANSACCIONES, "w") as archivo:
        for transaccion in transacciones:
            archivo.write(transaccion + "\n")

def validar_fecha(fecha):
    """
    Valida si la fecha ingresada tiene un formato válido (dd/mm/aaaa).
    """
    try:
        dia, mes, anio = map(int, fecha.split("/"))
        if len(str(anio)) != 4 or mes < 1 or mes > 12 or dia < 1 or dia > 31:
            return False
        return True
    except ValueError:
        return False

def main():
    while True:
        opcion = input("¿Qué deseas hacer? (agregar/listar/resumen/editar/eliminar/exportar/salir) ").lower()
        if opcion == "agregar":
            agregar_transaccion()
        elif opcion == "listar":
            listar_transacciones()
        elif opcion == "resumen":
            resumen_finanzas()
        elif opcion == "editar":
            editar_transaccion()
        elif opcion == "eliminar":
            eliminar_transaccion()
        elif opcion == "exportar":
            exportar_csv()
        elif opcion == "salir":
            break
        else:
            print("Opción inválida")

if __name__ == "__main__":
    main()

¿Qué deseas hacer? (agregar/listar/resumen/editar/eliminar/exportar/salir) agregar
Ingresa el tipo de transacción (gasto/ingreso): gasto
Ingresa la categoría (Comida, Transporte, Vivienda, Entretenimiento, Otros): comida
Ingresa una descripción para la transacción: transporte
Ingresa el monto de la transacción: 2000
Ingresa la fecha de la transacción (dd/mm/aaaa): 27/09/2024
Se ha registrado un gasto de 2000.0 en la categoría 'Comida' por 'transporte' el 27/09/2024
¿Qué deseas hacer? (agregar/listar/resumen/editar/eliminar/exportar/salir) exportar
Transacciones exportadas correctamente a 'transacciones.csv'
¿Qué deseas hacer? (agregar/listar/resumen/editar/eliminar/exportar/salir) listar
Transacciones registradas:
1. 27/09/2024 - Gasto de 2000.0 en 'Comida' por 'transporte'
¿Qué deseas hacer? (agregar/listar/resumen/editar/eliminar/exportar/salir) listar
Transacciones registradas:
1. 27/09/2024 - Gasto de 2000.0 en 'Comida' por 'transporte'
¿Qué deseas hacer? (agregar/listar/resumen/ed